# Convergence Rate for Pure, First-Order Feedback on SO(2) and SO(3)

For feedback driving from $R_0 \ne I$ to $I$, I would (naively) expected it to represent exponential
convergence of the angle around a given (constant) axis of rotation.

For $SO(2)$, there can only ever be one axis. For $SO(3)$, I would expect it to be the same axis as the one that can generate $R^a_0$.

FWIW: The following has convergence *bounds* regarding convergence of (second-order) feedback linearization on $SE(3)$:

[1] Lee, Taeyoung, Melvin Leok, and N. Harris McClamroch. \
&nbsp;&nbsp; “Control of Complex Maneuvers for a Quadrotor UAV Using Geometric Methods on SE(3).” arXiv, March 9, 2010. \
&nbsp;&nbsp; https://doi.org/10.48550/arXiv.1003.2005.

But would be nice to have an explicit solution for first-order feedback for both $SO(2)$ and $SO(3)$.

## Helper Stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (
    LeafSystem,
    Simulator,
    Integrator,
    DiagramBuilder,
)

In [ ]:
class SimpleVector(LeafSystem):
    def __init__(self, num_q, calc_qd):
        super().__init__()

        self.q = self.DeclareVectorInputPort("q", num_q)

        def calc_qd_sys(context, output):
            q = self.q.Eval(context)
            qd = calc_qd(q)
            output.set_value(qd)

        self.qd = self.DeclareVectorOutputPort("qd", num_q, calc_qd_sys)


def integrate(q0, calc_qd, tf):
    builder = DiagramBuilder()
    integ = builder.AddSystem(Integrator(num_q))
    q_actual_port = integ.get_output_port()
    controller = builder.AddSystem(SimpleVector(num_q, calc_qd))
    builder.Connect(q_actual_port, controller.q)
    builder.Connect(controller.qd, integ.get_input_port())

    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    integ_context = integ.GetMyContextFromRoot(diagram_context)
    controller_context = controller.GetMyContextFromRoot(diagram_context)

    integ.set_integral_value(integ_context, q_actual_init)

    ts = []
    q_as = []

    def monitor(diagram_context_in):
        t = diagram_context.get_time()
        q_actual = q_actual_port.Eval(integ_context)
        ts.append(t)
        q_as.append(q_actual)

    simulator = Simulator(diagram, diagram_context)
    simulator.set_monitor(monitor)
    simulator.AdvanceTo(tf)
    return ts, q_as

def maxabs(x):
    return np.max(np.abs(x))

In [ ]:
def rot(th):
    s = np.sin(th)
    c = np.cos(th)
    return np.array([
        [c, -s],
        [s, c],
    ])

def rot_jac(th):
    s = np.sin(th)
    c = np.cos(th)
    return np.array([
        [-s, -c],
        [c, -s],
    ])

def to_th(R):
    x = R[0, 0]
    y = R[1, 0]
    return np.arctan2(y, x)

def rot_err(R_actual, R_des):
    dR = R_actual @ R_des.T
    return to_th(dR)

def flatten(R):
    return R.flat[:]

def unflatten(q):
    return q.reshape((2, 2))

meh = np.array([[1, 2], [3, 4]])
assert (meh == unflatten(flatten(meh))).all()

## Sanity $\mathbb{R}^1$ Check

Dirt simple, coordinate $x$, with $x_0 != 0$, solution should be $x(t) = x_0 e^{-t}$.

In [ ]:
x0 = 1.0
q_actual_init = [x0]
num_q = len(q_actual_init)

def calc_x_expected(t):
    x = x0 * np.exp(-t)
    return x

def q_to_x(q):
    x, = q
    return x

def calc_qd(q_actual):
    x = q_to_x(q_actual)
    x_err = x
    xd = -x_err 
    qd_actual = [xd]
    return qd_actual

ts, q_as = integrate(q0=q_actual_init, calc_qd=calc_qd, tf=5.0)
x_as = [q_to_x(q_a) for q_a in q_as]
x_es = [calc_x_expected(t) for t in ts]
ts, x_as, x_es = map(np.asarray, (ts, x_as, x_es))

plt.plot(ts, x_es, "k--", linewidth=3, label="expected")
plt.plot(ts, x_as, "b-", label="actual")
plt.xlabel("t [s]")
plt.ylabel("th [rad]")
plt.legend()

## SO(2)

General rotation:

\begin{align}
    c = cos(\theta) \\
    s = sin(\theta) \\
    R(\theta) =
        \begin{bmatrix}
            c & -s \\
            s & c
        \end{bmatrix} \\
    \dot{R}(\theta) =
        \dot{\theta} \begin{bmatrix}
            -s & -c \\
            c & -s
        \end{bmatrix}
\end{align}

Define simple first-order feedback and expected closed-form solution:

\begin{align}
    \dot{\theta} = -\theta \\
    \theta(t) = \theta_0 e^{-t}
\end{align}

But for kicks and giggles, integrate via coordinate change as $R(t) = \int_0^t \dot{R}\left(\theta(R), \dot{\theta}\right) \ dt$

In [ ]:
theta0 = 1.0
R_actual_init = rot(theta0)
q_actual_init = flatten(R_actual_init)
num_q = len(q_actual_init)

def calc_th_expected(t):
    theta = theta0 * np.exp(-t)
    return theta

def q_to_th(q):
    R = unflatten(q)
    so2_tol = 1e-3
    so2_err = R @ R.T - np.eye(2)
    assert maxabs(so2_err) < so2_tol, so2_err
    return to_th(R)

def calc_qd(q_actual):
    R_actual = unflatten(q_actual)
    th_actual = to_th(R_actual)
    th_err = th_actual
    th_dot = -th_err
    Rd_actual = rot_jac(th_actual) * th_dot
    qd_actual = flatten(Rd_actual)
    return qd_actual

ts, q_as = integrate(q0=q_actual_init, calc_qd=calc_qd, tf=5.0)
th_as = [q_to_th(q_a) for q_a in q_as]
th_es = [calc_th_expected(t) for t in ts]
ts, th_as, th_es = map(np.asarray, (ts, th_as, th_es))

plt.plot(ts, th_es, "k--", linewidth=3, label="expected")
plt.plot(ts, th_as, "b-", label="actual")
plt.xlabel("t [s]")
plt.ylabel("th [rad]")
plt.legend()

## SO(3)

Use formulation as shown in [1], Eq. (10), but change order of multiplication so that we're computing the derivative
to align with the *world frame* angular velocity, not the *body frame* angular velocity.

Per that paper, let:

- $\hat{}: \mathbb{R}^3 \rightarrow so(3)$ be the operator produces skew matrices,
- $^\vee: so(3) \rightarrow \mathbb{R}^3$ be its inverse

First, restate the standard relationship between $R \in \mathbb{R}^{3 \times 3}$, its time derivatives,
and *world frame* angular velocities:

\begin{align}
    \dot{R} = {}^W\hat{\omega} R \\
    {}^W\hat{\omega} = \dot{R} R^T \\
\end{align}

Then we can get the following position-to-velocity/acceleration feedback:

\begin{align}
    R_\Delta &= R_a R_d^T \\
    \hat{e}_R &= \frac{1}{2} \left( R_\Delta - R_\Delta ^T \right)
\end{align}

Let $R(\lambda, \theta)$ provide us with angle-axis coordinates for rotation, $(\lambda, \theta)$.

First, let $R_0 = R(\lambda, \theta_0)$, for (what is hopefully a) constant $\lambda$ across all time.
Let $R_a = R$, and $R_d = I$.

Using this, we define the following first-order feedback and dynamics, with the expected (?!) solution:

\begin{align}
    \dot{R} &= -\hat{e}_R \\
    \theta(t) &= \theta_0 e^{-t} \\
    R(t) &= \left[ e^{-\hat{\lambda} \theta_0} \right] ^{-t} \\
        &= R_0 e^{-\hat{\lambda} \theta(t)} \hspace{10pt} \text{er is this even right?!}
\end{align}

Below, we plot the error / geodesic on $SO(3)$ as $d(R, I)$.